The inventory management ensures that the company always has the required materials and products in hand while keeping the cost as low as possible. Inventory Management refers to the process of supervising and controlling the stock items of a company. Typically, Inventory Management Systems are used by firms that either sell a product or manufacture a product for the purpose of accounting all the tangible goods that allow for a sale of a finished product, or parts for making a product. 

#### Steps to  recommend inventory per month for each product:
  As the data is having only 6 months we could not consider the timeseries forecasting with seasonality. Here we are using sampling distribution of means of frequency of products purchases each month to recommend inventory per month as Upper confidence intervel of 1.96sigma(95% confidence)limits

In [21]:
import numpy as np
import pandas as pd

In [92]:
tx_data = pd.read_table('C:\\Users\\Raghuram\\Desktop\\project\\000.txt',header=None,sep=",")

tx_data.columns=["date","cust_id","event","price","product_id","category"]
tx_data.head(5)


,date,cust_id,event,price,product_id,category
0,2019-01-14,007e188b-63b2-4c0b-9c74-2eb5a4cb1d9b,addToCart,337,PR923AT55PMADFMX,women>outerwear>vests
1,2019-01-14,007e188b-63b2-4c0b-9c74-2eb5a4cb1d9b,addToCart,337,PR923AT55PMADFMX,women>outerwear>vests
2,2019-01-14,007e188b-63b2-4c0b-9c74-2eb5a4cb1d9b,addToCart,337,PR923AT09IUWDFMX,women>outerwear>vests
3,2019-01-14,007e188b-63b2-4c0b-9c74-2eb5a4cb1d9b,addToCart,117,MA194AT06LOBDFMX,boys>topwear>sweatshirts
4,2019-01-14,007e188b-63b2-4c0b-9c74-2eb5a4cb1d9b,addToCart,117,MA194AT06LOBDFMX,boys>topwear>sweatshirts



# Extracting data related to purchase(event: buy)

  Given data is having 73495575 observations.For analysis extracting event 'buy' data having 9000+ observations

In [93]:
tx_data=tx_data.loc[tx_data['event'].isin(['buy'])]
print(tx_data.shape)
tx_data.head()

(9464, 6)


,date,cust_id,event,price,product_id,category
16796,2019-01-18,05fa0274-07ed-424f-9930-9e00a181a57b,buy,1999,TI778SH75SKEDFMX,men>shoes>boots
16797,2019-01-18,05fa0274-07ed-424f-9930-9e00a181a57b,buy,1499,TI778SH87SJSDFMX,men>shoes>boots
16798,2019-01-11,0cbc10e5-005a-4960-a7fb-28931bd82daf,buy,1199,TH934AT94SAVDFMX,women>outerwear>jackets
16799,2019-01-18,1256d1c8-feae-4226-871a-b90e7d918ee0,buy,299,SQ017AT94NEVDFMX,men>activewear>tshirts
16800,2019-01-18,1256d1c8-feae-4226-871a-b90e7d918ee0,buy,299,DO528AB27RRWDFMX,men>bottomwear>shorts


We have no. of customers who puchases products are 2819
Total no. of unique products ordered are 3656

In [94]:
cust_count=tx_data['cust_id'].value_counts()
prod_count=tx_data['product_id'].value_counts()
print(cust_count.head())
print(prod_count.head())
print(cust_count.shape)
prod_count.shape

50bb9727-5420-42a1-8605-73a5042d1c8b    125
223c59ce-99fc-4e0d-87a9-4156c87fd5d2    114
4d416d08-e2ef-4c1c-af4e-db84d78dc8a9     56
e76aff10-b325-46dc-949a-29848e1baef7     40
85bb6a6b-ae70-487b-a758-c93a54da0426     38
Name: cust_id, dtype: int64
TI778SH85SJUDFMX    129
TI778SH80SJZDFMX     69
TI778SH71SKIDFMX     38
ST089SH56SKXDFMX     35
TI778SH84SJVDFMX     26
Name: product_id, dtype: int64
(2819,)


(3656,)

In [95]:
tx_data.drop(["event","category","price"],axis=1,inplace=True)

In [96]:
import datetime
#converting the type of  Date Field from string to date.
tx_data['date'] = pd.to_datetime(tx_data['date'])

#creating day,Month field for the ease of reporting and visualization
tx_data['date_day']=tx_data['date'].dt.day
tx_data['Month'] = tx_data['date'].dt.month

In [97]:
tx_data.reset_index()
tx_data.head()

,date,cust_id,product_id,date_day,Month
16796,2019-01-18,05fa0274-07ed-424f-9930-9e00a181a57b,TI778SH75SKEDFMX,18,1
16797,2019-01-18,05fa0274-07ed-424f-9930-9e00a181a57b,TI778SH87SJSDFMX,18,1
16798,2019-01-11,0cbc10e5-005a-4960-a7fb-28931bd82daf,TH934AT94SAVDFMX,11,1
16799,2019-01-18,1256d1c8-feae-4226-871a-b90e7d918ee0,SQ017AT94NEVDFMX,18,1
16800,2019-01-18,1256d1c8-feae-4226-871a-b90e7d918ee0,DO528AB27RRWDFMX,18,1


In [219]:
tx_products=tx_data.groupby(['Month','product_id']).size().reset_index(name="freq")
#tx_products.to_csv('C:\\Users\\Raghuram\\Desktop\\Python Project customer analytics\\inventory_data.csv')
tx_products.head()

,Month,product_id,freq
0,1,AR811AB00DCDDFMX,1
1,1,AR811AB06DBXDFMX,1
2,1,AR811AB12DBRDFMX,1
3,1,AR811AB15DBODFMX,3
4,1,AR811AB16DBNDFMX,1


##### Here the resultant data frame gives us the inventory stock recommendation of each product per month with 95% confidence

In [261]:
products=tx_data['product_id'].unique()
UCL=[]
for prod in products:
    sample_mean=[]
    for n in range(1,50):
        sample_mean.append(np.random.choice(tx_products.loc[tx_products['product_id']==prod,'freq'],10,replace=True).mean())
    SDMean=np.mean(sample_mean)
    stdev=np.std(tx_products.loc[tx_products['product_id']==prod,'freq'])
    UCL.append(round(SDMean+1.96*stdev/np.sqrt(tx_products.loc[tx_products['product_id']==prod,'freq'].count())))
UCL

[3.0,
 6.0,
 2.0,
 4.0,
 9.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 6.0,
 4.0,
 2.0,
 2.0,
 2.0,
 3.0,
 3.0,
 5.0,
 2.0,
 2.0,
 2.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 7.0,
 2.0,
 7.0,
 3.0,
 1.0,
 2.0,
 2.0,
 1.0,
 3.0,
 4.0,
 1.0,
 1.0,
 2.0,
 5.0,
 2.0,
 1.0,
 2.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 2.0,
 2.0,
 2.0,
 6.0,
 1.0,
 1.0,
 2.0,
 2.0,
 1.0,
 11.0,
 2.0,
 4.0,
 1.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 3.0,
 1.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 8.0,
 2.0,
 1.0,
 1.0,
 1.0,
 3.0,
 1.0,
 3.0,
 2.0,
 1.0,
 1.0,
 2.0,
 2.0,
 3.0,
 1.0,
 4.0,
 4.0,
 1.0,
 1.0,
 1.0,
 3.0,
 1.0,
 2.0,
 2.0,
 2.0,
 2.0,
 1.0,
 3.0,
 1.0,
 1.0,
 2.0,
 4.0,
 1.0,
 5.0,
 2.0,
 5.0,
 3.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 2.0,
 2.0,
 5.0,
 1.0,
 1.0,
 26.0,
 5.0,
 3.0,
 3.0,
 5.0,
 10.0,
 1.0,
 7.0,
 5.0,
 2.0,
 3.0,
 3.0,
 1.0,
 3.0,
 3.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 38.0,
 1.0,
 1.0,
 2.0,
 1.0,
 2.0,
 2.0,
 2.0,
 1.0,
 1.0,
 2.0,
 1.0,
 1.0,
 1.0,


In [319]:
UCL_df=pd.DataFrame(data={'UCL':UCL})
products_inventory=pd.DataFrame(data={'product_id':products})
products_inventory=pd.concat([products_inventory,UCL],axis=1)
products_inventory.head(10)

,product_id,UCL
0,TI778SH75SKEDFMX,3.0
1,TI778SH87SJSDFMX,6.0
2,TH934AT94SAVDFMX,2.0
3,SQ017AT94NEVDFMX,4.0
4,DO528AB27RRWDFMX,9.0
5,PH184AT97GFYDFMX,1.0
6,GO016AT78BJTDFMX,1.0
7,ZO302AT17BAODFMX,1.0
8,ZO302AT18BANDFMX,1.0
9,GO016AT39BLGDFMX,1.0
